# Technical Setup



## Import of Demo Translations
In the first step we import the given translations as pandas Dataframes and print a quick overview of the dataframe.


In [ ]:
import pickle

from mlflow import transformers

with open('machine_translation.pkl', 'rb') as f:
    # Einlesen des Dataframes
    data = pickle.load(f)

print(data)

## Import of AI-Models
In the second step we import the AI-Models which are given in the specified task. For doing so we use the `llama-cpp-python` library (further documentation can be found [here](https://github.com/abetlen/llama-cpp-python)) and import the models directly from [huggingface](https://huggingface.co/).

Quick overview and installation guide of llama.cpp: 
- https://www.datacamp.com/tutorial/llama-cpp-tutorial
- https://christophergs.com/blog/running-open-source-llms-in-python



In [ ]:
from llama_cpp import Llama

gemma = Llama.from_pretrained(
	repo_id="lmstudio-ai/gemma-2b-it-GGUF",
	filename="gemma-2b-it-q8_0.gguf",
    n_gpu_layers=1,
    verbose=False,
)


In [ ]:
llama32 = Llama.from_pretrained(
	repo_id="hugging-quants/Llama-3.2-3B-Instruct-Q8_0-GGUF",
	filename="llama-3.2-3b-instruct-q8_0.gguf",
    n_gpu_layers=1,
    verbose=False,)


In [ ]:
llama31 = Llama.from_pretrained(
	repo_id="lmstudio-community/Meta-Llama-3.1-8B-Instruct-GGUF",
	filename="Meta-Llama-3.1-8B-Instruct-Q5_K_M.gguf",
    verbose=False,)


In [ ]:
aya23 = Llama.from_pretrained(
	repo_id="bartowski/aya-23-35B-GGUF",
	filename="aya-23-35B-Q5_K_M.gguf",
    n_gpu_layers=1,
    verbose=False,
)


In [ ]:
output = llama31(
	"Q: Übersetze mir folgenden Text ins Englische: 'Hallo ich heiße Peter.'",
	max_tokens=512,
	echo=False,
    stop=["Q:"]
)

# print(output)
print(output['choices'][0]['text'])

In [ ]:
output2 = aya23(
	"Q: Übersetze mir folgenden Text ins Englische: 'Hallo ich heiße Peter.'",
	max_tokens=512,
	echo=False,
    stop=["Q:"]
)

# print(output)
print(output2['choices'][0]['text'])

In [ ]:
print(output)

In [ ]:
import mlflow

mlflow.set_tracking_uri("http://127.0.0.1:5000")
mlflow.set_experiment("check-localhost-connection")

with mlflow.start_run():
    mlflow.log_metric("foo", 1)
    mlflow.log_metric("bar", 3)

## MetricX Score berechnen
https://github.com/google-research/metricx

In [1]:
import sys
from metricx24 import predict

# Simuliere Kommandozeilenargumente
sys.argv = [
    "predict.py",  # Name des Skripts (wird ignoriert)
    "--tokenizer", "google/mt5-xl",
    "--model_name_or_path", "google/metricx-24-hybrid-large-v2p6-bfloat16",
    "--max_input_length", "1536",
    "--batch_size", "1",
    "--input_file", "./input.jsonl",
    "--output_file", "./output.jsonl"
]

# Rufe die main-Funktion auf
predict.main()

print("Vorhersage abgeschlossen. Ergebnisse sind in 'output.jsonl' gespeichert.")

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
/opt/anaconda3/envs/softwarequality/lib/python3.12/site-packages/transformers/convert_slow_tokenizer.py:561: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(
Passing a tuple

Vorhersage abgeschlossen. Ergebnisse sind in 'output.jsonl' gespeichert.


In [2]:
import subprocess
import json

data = [
    {"id": "1", "source": "Ich lerne Python für Machine Learning.", "reference": "I am learning Python for machine learning.", "hypothesis": "I'm going to the supermarket today."}
]

# JSONL-Datei schreiben mit doppelten Anführungszeichen
with open("input.jsonl", "w", encoding="utf-8") as f:
    for entry in data:
        json.dump(entry, f)  # Wandelt das Dictionary in einen JSON-kompatiblen String um
        f.write("\n")  # Zeilenumbruch nach jedem JSON-Objekt

print("Die Datei 'input.jsonl' wurde erfolgreich erstellt!")


# Definiere den Befehl als Liste von Argumenten
command = [
    "python", "-m", "metricx24.predict",
    "--tokenizer", "google/mt5-xl",
    "--model_name_or_path", "google/metricx-24-hybrid-large-v2p6-bfloat16",
    "--max_input_length", "1536",
    "--batch_size", "1",
    "--input_file", "./input.jsonl",
    "--output_file", "./output.jsonl"
]

# Subprocess ausführen und die Ausgabe live anzeigen
process = subprocess.Popen(
    command,
    stdout=subprocess.PIPE,  # Standardausgabe abfangen
    stderr=subprocess.PIPE,  # Fehlerausgabe abfangen
    text=True,               # Ausgabe als Text (kein Byte-Objekt)
    bufsize=1,               # Zeilenweise Puffern
    universal_newlines=True
)

# Zeilenweise Ausgabe lesen und live anzeigen
print("Live-Ausgabe des Befehls:\n")
for line in process.stdout:
    print(line, end="")  # Zeilen ohne zusätzlichen Zeilenumbruch ausgeben

# Fehler ausgeben (falls vorhanden)
for line in process.stderr:
    print(f"ERROR: {line}", end="")

# Auf den Abschluss des Prozesses warten
process.wait()
print("\nBefehl abgeschlossen.")

Die Datei 'input.jsonl' wurde erfolgreich erstellt!
Live-Ausgabe des Befehls:



python(10256) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


ERROR: You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
ERROR: /opt/anaconda3/envs/softwarequality/lib/python3.12/site-packages/transformers/convert_slow_tokenizer.py:561: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
ERROR:   warnings.